In [1]:
import geopandas as gpd
import folium
from folium import plugins
import branca
import time
import os
start_time = time.time()

In [2]:
print(gpd.__version__)
print(folium.__version__)
print(branca.__version__)

0.8.1
0.11.0
0.4.1


In [3]:
os.chdir('..')

In [5]:
## Load geojson as geopandas dataframe
dc_gdf = gpd.read_file("data\\processed\\zillow_nb_dc_merged.geojson")    

In [6]:
centroid=dc_gdf.geometry.centroid ## identifies the center point of all the neighborhood shapes 

m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=12) ## initiaes a map based on the centroid
    
m

<ipython-input-6-a65073720761>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid=dc_gdf.geometry.centroid ## identifies the center point of all the neighborhood shapes


In [7]:
variable = 'active_rest' #active restaurants in Washington, DC as of 9/2019
name = '# Active Restaurants'

print(name, "colorscale")
print("Min:",dc_gdf[variable].min())
print("Max:",dc_gdf[variable].max())
    
colorscale = branca.colormap.linear.YlOrRd_09.scale(dc_gdf[variable].min(), dc_gdf[variable].max()) 
colorscale

# Active Restaurants colorscale
Min: 0.0
Max: 294.0


In [8]:
# create df with neighborhood name and variable of interest, sorted from largest to smallest
df = dc_gdf[['name', variable]].sort_values(by = variable, ascending = False) 
    
# reset index so that the largest value corresponds to row 0 and smallest to row 136
df.reset_index(inplace = True)
leg_brks = list(df[df.index.isin([0,4,9,19,29,49])][variable]) # identify the value of the var by index position
    
# make the smallest value of the scale be 0
leg_brks.append(0)
leg_brks.sort() # sort from smallest to largest
print("Quantiles:", leg_brks)
    

Quantiles: [0, 6.0, 16.0, 27.0, 64.0, 104.0, 294.0]


In [9]:
print(name, "colorscale")

colorscale = branca.colormap.linear.YlOrRd_09.scale(dc_gdf[variable].min(), dc_gdf[variable].max()) 
colorscale = colorscale.to_step(n = 6, quantiles = leg_brks) ## sets quantile breaks 
colorscale.caption = name ## adds name for legend
    
colorscale

# Active Restaurants colorscale


In [10]:
variable = 'active_rest' #active restaurants in Washington, DC as of 9/2019
name = '# Active Restaurants'

folium.GeoJson(dc_gdf, ## GeoPandas dataframe
               name="Washington DC",
                   
               ## controls the fill of the geo regions; applying colorscale based on variable
               style_function=lambda x: {"weight":1
                                         , 'color': '#545453'
                                         ## if variable is 0 map is a very light grey
                                         ## else colorscale applies based on variable
                                         , 'fillColor':'#9B9B9B' if x['properties'][variable] == 0 
                                         else colorscale(x['properties'][variable])
                                         ## similarly opacity is increased if value is 0
                                         , 'fillOpacity': 0.2 if x['properties'][variable] == 0 
                                         else 0.5},
                   
               ## changes styling of geo regions upon hover
               highlight_function=lambda x: {'weight':3, 'color':'black', 'fillOpacity': 1}, 
               
                ## tooltip can include information from any column in the GeoPandas dataframe   
                tooltip=folium.features.GeoJsonTooltip(
                fields=['name', 'active_tot', 'active_tot_sq_mi', variable],
                aliases=['Neighborhood:', '# Active Restaurants + Delis:', '# Active Per Sq Mi', name])
              ).add_to(m)

## add colorscale to map so that it appears as the legend
colorscale.add_to(m)
    
m